In [1]:
# Representing variables or values constructed with variables
abstract type SymVal end

# A variable with name S and value x
struct Var{S} <: SymVal
    x::Float64
end

Var(s::Symbol, x) = Var{s}(Float64(x))

# This stores how a value was created
struct Operation{FuncType,ArgTypes}
    op::FuncType
    args::ArgTypes
end

# A value created using Variables
struct Value{OT} <: SymVal
    x::Float64
    op::OT
end

value(var::Var) = var.x
value(val::Value) = val.x
value(x::Number) = x

import Base.show
Base.show(io::IO, var::Var{S})  where S= print(io, S, ": ", value(var))
Base.show(io::IO, val::Value) = print(io, "Value ", value(val))

# dx/dx = 1
diff(x::Var{S}, y::Var{S}) where S = 1.0

# dx/dy = 0   if x,y are independent
diff(x::Var{S}, y::Var{V}) where {S,V} = 0.0

# d5/dx = 0 / deriving a constant is always zero
diff(x::Number, _whatever ) = 0.0

# ---------------------- Buisness logic below here--------------------------------------
import Base.+
+(val1::SymVal, val2::SymVal) = Value(value(val1) + value(val2), Operation(+,(val1,val2)))
+(val1::SymVal, val2) = Value(value(val1) + value(val2), Operation(+,(val1,val2)))
+(val1, val2::SymVal) = Value(value(val1) + value(val2), Operation(+,(val1,val2)))

# ok, this is getting annoying. Lets use metaprogramming instead to define the rest of 
# the operators for me
for op in (:*, :-, :/)
    @eval begin
        import Base.$op
        # these lines do exactly the same as the three above for +, but now I apply the same template to
        # all operators in the list at once.
        # I don't expect a newbie to julia to understand how this works, just to showcase how the language can
        # save you work.
        # Also: Githubs notebook diplay does really not play nice with the $ sign, you might need to download this to see whats happening
        $op(val1::SymVal, val2::SymVal) = Value($op(value(val1), value(val2)), Operation($op,(val1,val2)))
        $op(val1::SymVal, val2) = Value($op(value(val1), value(val2)), Operation($op,(val1,val2))) 
        $op(val1, val2::SymVal) = Value($op(value(val1), value(val2)), Operation($op,(val1,val2)))
    end        
end

import Base.exp
exp(x::SymVal) = Value(exp(value(x)), Operation(exp, x))
# lets do the same for the other single-arg functions
for fun in ( :log, :sin, :cos, :tan)
    @eval begin
        import Base.$fun
        # this looks like the line with exp above, but using all the other functions
        $fun(x::SymVal) = Value($fun(value(x)), Operation($fun, x))
    end
end

# d(f+g)/dx = df/dx + dg/dx
diff(x::Value{Operation{FT, AT}}, var::Var) where {FT <: typeof(+), AT} = sum(diff(y, var) for y in x.op.args)

# differentiation of a product. It's just chain rule
# this gracelessly fails if op.args is not a two-tuple
diff(x::Value{Operation{FT, AT}}, var::Var) where {FT <: typeof(*), AT} = 
    diff(x.op.args[1], var) * value(x.op.args[2]) + diff(x.op.args[2], var) * value(x.op.args[1]) 

# this is a demonstration, so I'll let you try to implement the other binary operators /, - etc yourself
# maybe you have a binary operator that I forgot? See wether you can add it to the system, and how to do
# it most efficiently and most compiler-friendly

# lets do all the single-arg functions in one go, using the chain rule!
# elemDiff(FT) returns f',    x.op.args is g(x) 
# d(f(g(x)))/dx = g'(x) * f'(g(x))
diff(x::Value{Operation{FT, AT}}, var::Var) where {FT, AT <: Union{Number, SymVal}} = diff(x.op.args, var) * elemDiff(FT)(value(x.op.args))


# this function defines the elementary differentials for one-argument functions.
# note that it works on the type-level
elemDiff(::Type{typeof(exp)}) = exp
elemDiff(::Type{typeof(sin)}) = cos
elemDiff(::Type{typeof(cos)}) = x -> -1 * sin(x)
# .. and so on...

elemDiff (generic function with 3 methods)

In [2]:
x = Var(:x, 2)

x: 2.0

In [3]:
y = Var(:y, 3)

y: 3.0

In [4]:
x + x 

Value 4.0

In [5]:
x + x + y + 2.4

Value 9.4

Basic calculation seems to work...

In [6]:
diff(x + x, x)

2.0

$\frac{d(x+x)}{dx} = 2$, so far so good

In [7]:
sin(x)

Value 0.9092974268256817

In [8]:
sin(2)

0.9092974268256817

In [9]:
diff(sin(x), x)

-0.4161468365471424

In [10]:
cos(2)

-0.4161468365471424

In [11]:
sin(exp(x))

Value 0.8938549549128102

In [12]:
sin(exp(2))

0.8938549549128102

In [13]:
diff(sin(exp(x)), x)

3.312929423104333

In [14]:
cos(exp(x)) * exp(x)

Value 3.312929423104333

In [15]:
diff(exp(x*x), x)

218.39260013257694

In [16]:
 (x -> 2*x*exp(x^2) )(2)

218.39260013257694

so far so good, but how fast are we, and how good can the compiler optimize this?

To find out, we can use `@code_llvm` which will give us the compiled low-level code
of the called function. I do not expect anyone to understand much llvm code,
but the ones we will look at are really simple.

In [17]:
@code_llvm diff(x + x  , x)

;  @ In[1]:73 within `diff`
define double @julia_diff_1338({ double, { [2 x [1 x double]] } }* nocapture noundef nonnull readonly align 8 dereferenceable(24) %0, [1 x double]* nocapture noundef nonnull readonly align 8 dereferenceable(8) %1) #0 {
top:
  ret double 2.000000e+00
}


The super long line after the `define` is just a function header, the body is the code after the `top:`

As you can see, the function does nothing, it just returns 2.0; The compiler knew the result at compiletime.

In [18]:
@code_llvm diff(x + x + x, x)

;  @ In[1]:73 within `diff`
define double @julia_diff_1365({ double, { { { double, { [2 x [1 x double]] } }, [1 x double] } } }* nocapture noundef nonnull readonly align 8 dereferenceable(40) %0, [1 x double]* nocapture noundef nonnull readonly align 8 dereferenceable(8) %1) #0 {
top:
  ret double 3.000000e+00
}


Now it statically returns 3

In [19]:
@code_llvm  diff( 2.0 * x, x)

;  @ In[1]:77 within `diff`
define double @julia_diff_1369({ double, { { double, [1 x double] } } }* nocapture noundef nonnull readonly align 8 dereferenceable(24) %0, [1 x double]* nocapture noundef nonnull readonly align 8 dereferenceable(8) %1) #0 {
top:
; ┌ @ In[1]:8 within `value`
; │┌ @ Base.jl:37 within `getproperty`
    %2 = getelementptr inbounds { double, { { double, [1 x double] } } }, { double, { { double, [1 x double] } } }* %0, i64 0, i32 1, i32 0, i32 1, i64 0
; └└
; ┌ @ float.jl:410 within `*`
   %3 = load double, double* %2, align 8
   %4 = fmul double %3, 0.000000e+00
; └
; ┌ @ tuple.jl:29 within `getindex`
   %5 = getelementptr inbounds { double, { { double, [1 x double] } } }, { double, { { double, [1 x double] } } }* %0, i64 0, i32 1, i32 0, i32 0
; └
; ┌ @ float.jl:410 within `*`
   %6 = load double, double* %5, align 8
; └
; ┌ @ float.jl:408 within `+`
   %7 = fadd double %4, %6
; └
  ret double %7
}


Here, the compiler can no longer statically return 2, since the factor 2 is not present in the type signature. 

Still, if you look at whats happening here, there is nothing but a few simple floating point lookups and a multiply.

Btw, if you have trouble reading this, all the non-colorful lines are basically comments and can be ignored

In [20]:
@code_llvm diff(sin(x) , x)

;  @ In[1]:87 within `diff`
define double @julia_diff_1371({ double, { [1 x double] } }* nocapture noundef nonnull readonly align 8 dereferenceable(16) %0, [1 x double]* nocapture noundef nonnull readonly align 8 dereferenceable(8) %1) #0 {
top:
; ┌ @ In[1]:8 within `value`
; │┌ @ Base.jl:37 within `getproperty`
    %2 = getelementptr inbounds { double, { [1 x double] } }, { double, { [1 x double] } }* %0, i64 0, i32 1, i32 0, i64 0
; └└
  %3 = load double, double* %2, align 8
  %4 = call double @j_cos_1373(double %3) #0
  ret double %4
}


This _looks_ a bit more complicated, but all thats happening here really is that we are fetching some double value, and then passing it to the `cos` function

In [21]:
@code_llvm diff(sin(x*x) , x)

;  @ In[1]:87 within `diff`
define double @julia_diff_1376({ double, { { double, { [2 x [1 x double]] } } } }* nocapture noundef nonnull readonly align 8 dereferenceable(32) %0, [1 x double]* nocapture noundef nonnull readonly align 8 dereferenceable(8) %1) #0 {
top:
;  @ In[1]:87 within `diff` @ In[1]:77
; ┌ @ In[1]:8 within `value`
; │┌ @ Base.jl:37 within `getproperty`
    %2 = getelementptr inbounds { double, { { double, { [2 x [1 x double]] } } } }, { double, { { double, { [2 x [1 x double]] } } } }* %0, i64 0, i32 1, i32 0, i32 1, i32 0, i64 1, i64 0
; └└
; ┌ @ float.jl:410 within `*`
   %3 = load double, double* %2, align 8
; └
; ┌ @ In[1]:8 within `value`
; │┌ @ Base.jl:37 within `getproperty`
    %4 = getelementptr inbounds { double, { { double, { [2 x [1 x double]] } } } }, { double, { { double, { [2 x [1 x double]] } } } }* %0, i64 0, i32 1, i32 0, i32 1, i32 0, i64 0, i64 0
; └└
; ┌ @ float.jl:410 within `*`
   %5 = load double, double* %4, align 8
; └
; ┌ @ float.jl:408 wi

...And here we fetch the value of `x`, calculate `x+x`, fetch the stored value of `x^2`, calculate `cos(x^2)`, and then multiply the two values. So basically $(x+x) * cos(x^2)$

(Actually, we fetch `x` twice. The type signature does not contain the information that the two values are identical)

In [22]:
@code_llvm diff(cos(x) , x)

;  @ In[1]:87 within `diff`
define double @julia_diff_1381({ double, { [1 x double] } }* nocapture noundef nonnull readonly align 8 dereferenceable(16) %0, [1 x double]* nocapture noundef nonnull readonly align 8 dereferenceable(8) %1) #0 {
top:
; ┌ @ In[1]:8 within `value`
; │┌ @ Base.jl:37 within `getproperty`
    %2 = getelementptr inbounds { double, { [1 x double] } }, { double, { [1 x double] } }* %0, i64 0, i32 1, i32 0, i64 0
; └└
; ┌ @ In[1]:94 within `#11`
   %3 = load double, double* %2, align 8
   %4 = call double @j_sin_1383(double %3) #0
; │┌ @ promotion.jl:411 within `*` @ float.jl:410
    %5 = fneg double %4
; └└
  ret double %5
}


This fetches a double, passes it to the `sin` function, and then flips the sign of the result; $-sin(x)$

## Final Notes

of course I chose some rather benign examples. There are limitations and shortcomings with this approach. But I hope I could show how one can engrave information into the type signature of their values, and how to use such information to controll behaviour and have the compiler do my work as a programmer for me at compiletime.
